<img align='left' src = https://www.linea.org.br/brand/linea-logo-color.svg width=150 style='padding: 20px'> 

## DESI EDR Spectroscopic Redshifts

Contact: Julia Gschwend ([julia@linea.org.br](mailto:julia@linea.org.br))
<br>
<br>


<font color='red' SIZE=4> DISCLAIMER: THIS IS NOT AN OFFICIAL DATA RELEASE. <br>
This notebook was created to test the PZ Server under development </font>

### Intro 

This notebook contains a brief characterization of the spectroscopic _redshifts_ (spec-z) measurements in the DESI EDR catalog, which has been publicly distributed and described in detail in scientific literature by DESI project.

The data presented here were obtained from the [DESI data release web page](https://data.desi.lbl.gov/doc/releases/edr/vac/zcat/) and filtered to contain only objects classified as **galaxies** and with `ZCAT_PRIMARY = True`. Be aware that not all measurements of the original catalog are present here.

|   |Number of redshifts|
|---|--:|
|this sample|1,591,474|
|original catalog | 2,451,325|  


## Reference paper

[DESI Collaboration et al. (2023b), The Early Data Release of the Dark Energy Spectroscopic Instrument](https://arxiv.org/abs/2306.06308)


See also the [DESI Data License and Acknowledgments policy on the data release website](https://data.desi.lbl.gov/doc/acknowledgments/).  
  


--- 

## Sample characterization



Imports and configs

In [ ]:
# PZ Server
from pzserver import PzServer

In [ ]:
# General
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys

# Astropy
from astropy import units as u
from astropy.coordinates import SkyCoord


# Bokeh, HoloViews
import bokeh
import holoviews as hv
from holoviews import opts

# Config
import warnings
warnings.filterwarnings('ignore')
%reload_ext autoreload 
%autoreload 2 
%matplotlib inline 
plt.rcParams.update({'figure.max_open_warning': 0})
hv.extension('bokeh')
#output_notebook()

Getting authorization for PZ Server via API Token:

In [ ]:
with open('../token.txt', 'r') as file:
    token = file.read()
pz_server = PzServer(token=token, host="pz-dev") # "pz-dev" is the temporary host for test phase  

Printing the libraries versions:

In [ ]:
print('Python version: ' + sys.version)
print('Numpy version: ' + np.__version__)
print('HoloViews version: ' + hv.__version__)

In [ ]:
def fmt(x):
    return '{:.1f}%'.format(x)

Read spec-z catalog file

In [ ]:
catalog = pz_server.get_product('148_desi_edr')

In [ ]:
catalog

In [ ]:
catalog.display_metadata()

In [ ]:
specz_catalog = catalog.data

In [ ]:
type(specz_catalog)

In [ ]:
assert len(specz_catalog) == 1_591_474

In [ ]:
specz_catalog.info(memory_usage="deep")

In [ ]:
specz_catalog.head()

Meaning of main columns:

| Column name | Meaning |
|--:|:--|
| **TARGET_RA** | Right Ascension (degrees) |
| **TARGET_DEC** | Declination (degrees) |
| **Z** | Redshift |
| **ZERR** | Redshift error |

Basic statistics

In [ ]:
specz_catalog.describe()

In [ ]:
#frac = 0.02
#spec_sample_for_plots = specz_catalog.sample(frac=frac, axis='index')
#assert len(spec_sample_for_plots) == round(frac * len(specz_catalog))
#print(len(spec_sample_for_plots))
specz_catalog.rename(columns={"TARGET_RA":"ra", "TARGET_DEC":"dec"}, inplace=True) 
spec_sample_for_plots = specz_catalog # comment this line to use a fraction of the data


In [ ]:
spec_sample_for_plots

--- 

#### Spatial distribution


In [ ]:
coords = SkyCoord(ra=-np.array(spec_sample_for_plots.ra)*u.degree, 
                  dec=np.array(spec_sample_for_plots.dec)*u.degree, frame='icrs')
spec_sample_for_plots.ra = np.array(coords.ra.wrap_at(180*u.degree))
spec_sample_for_plots.dec = np.array(coords.dec)

In [ ]:
%%time
fig = plt.figure(figsize=[14,6])
ax = fig.add_subplot(111, projection='mollweide')   
ra_rad = coords.ra.wrap_at(180 * u.deg).radian
dec_rad = coords.dec.radian
plt.plot(ra_rad, dec_rad, '.', alpha=0.3)
org=0.0
tick_labels = np.array([150, 120, 90, 60, 30, 0, 330, 300, 270, 240, 210])
tick_labels = np.remainder(tick_labels+360+org,360)
ax.set_xticklabels(tick_labels)     # we add the scale on the x axis
ax.set_xlabel('R.A.')
ax.xaxis.label.set_fontsize(14)
ax.set_ylabel('Dec.')
ax.yaxis.label.set_fontsize(14)
ax.grid(True)
plt.tight_layout()

Redshift distribution

In [ ]:
redshift = hv.Dimension('z', label='spec-z')#, range=(0.0, 2.0))
(count, z_bin) = np.histogram(spec_sample_for_plots.Z, bins='fd')

In [ ]:
z_distribution = hv.Histogram((count, z_bin), kdims=redshift).opts(
    title='Redshift distrbution', xlabel='spec-z', height=400, width=800,  xlim=(0.,2.))   
z_distribution